# Assignment 1d Notebook: Multi-Objective EA
This notebook is the final notebook in assignment series 1 and further iterates on the progress you've made so far. This notebook will guide you through a novel implementation of a multi-objective EA (MOEA) that utilizes the EA framework you've been implementing since Assignment 1b as much as possible. Similar to the previous assignments, you should copy over the following files:
* 1a_notebook.ipynb
* 1b_notebook.ipynb
* 1c_notebook0.ipynb
* 1c_notebook1.ipynb
* baseEvolution.py
* binaryGenotype.py
* selection.py
* selfAdaptiveEvolution.py
* selfAdaptiveGenotype.py


*Be careful* to not copy over functions relating to the provided fitness functions, GPac, and static agents. We have updated `fitness.py` for this assignment, so be particularly careful not to accidentally copy the old version from Assignment 1c.

As usual, be sure to **read all of this notebook** and you can start by executing the next cell.

In [ ]:
# Configure this notebook to automatically reload modules as they're modified
# https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# define utility plotting function
def plotMap(maze):
    '''A so-so map plotting function'''
    pltmaze = [[None for __ in range(len(maze))] for _ in range(len(maze[0]))]
    for y in range(len(maze[0])):
        for x in range(len(maze)):
            pltmaze[y][x] = 0 if maze[x][y]==1 else 1
    plt.matshow(pltmaze, origin='lower')
    plt.xticks(range(len(pltmaze[0])))
    plt.yticks(range(len(pltmaze)))
    plt.gca().set_xticks([x - 0.5 for x in plt.gca().get_xticks()][1:], minor='true')
    plt.gca().set_yticks([y - 0.5 for y in plt.gca().get_yticks()][1:], minor='true')
    plt.grid(which='minor')
    plt.show()

print('The first cell has been executed!')

## Multi-Objective Fitness Evaluations
In a traditional MOEA implementation like the Non-dominated Sorting Genetic Algorithm II [(NSGA-II)](https://ieeexplore.ieee.org/abstract/document/996017?casa_token=sIy9DHU74qAAAAAA:f9M0Nu6WrHIswdRILFlqhxUqW-rK1nfke65Xw88A1JNX5TaaXZAL76yrC3L8WncdUlrDi25Y7Zo), individuals in a population are assessed by multiple objectives, assigned a score for each objective, and are then sorted into levels of non-domination (more on that later) during parent and survival selection. In a traditional MOEA implementation, you might write a multi-objective version of k-tournament that samples k individuals from the population and selects the winner as the individual who is on the Pareto front from those k individuals. This formulation thus doesn't use a single-value notion of fitness as your existing k-tournament implementations do.

To enable the re-use of your existing selection algorithms, we propose a novel MOEA implementation that assigns individuals in a population a singular fitness value based on their level of non-domination. This behaves in a functionally equivalent manner in the rank-based selection algorithms used in traditional MOEA implementations. This also lets you use non-traditional selection algorithms like Fitness Proportional Selection in your MOEA, though they may perform poorly.

In your MOEA for this assignment, you will do the following:
* Evaluate new members in a population to assign objective scores
* Construct a domination table for *all* individuals in the population
* Sort the population into levels of non-domination where level 0 is the Pareto front
* To each individual in the population, assign a representative fitness that is inversely proportional to their level of non-domination 
 - i.e., `num_levels - my_level`

To accomplish this, you'll implement three functions:
1. `evaluate_popluation` - which assigns objective score to new population members after generation
2. `dominates` - which takes individuals `A` and `B` as  inputs and returns `True` if `A` dominates `B`
3. `non_domination_sort` - which sorts the population into levels of non-domination and assigns a representative fitness.

First, let's start by showing an example of the multi-objective fitness function.

In [ ]:
from selection import *
from snakeeyes import readConfig
from binaryGenotype import binaryGenotype
from fitness import repair_and_test_map, translate_gene, repair_map

config = readConfig('./configs/green1d_config.txt', globalVars=globals(), localVars=locals())

testSolution = binaryGenotype()
testSolution.randomInitialization(**config['initialization_kwargs'])

testSolution.objectives, testSolution.log = repair_and_test_map(testSolution.gene, **config['fitness_kwargs'])
# testSolution.rawFitness = testSolution.objectives[0]

game_log_path = 'worldFiles/1dnotebooktest.txt'
with open(game_log_path, 'w') as f:
	[f.write(f'{line}\n') for line in testSolution.log]
    
print(f"The solution's objective scores are {testSolution.objectives} and the game log was written to {game_log_path}")
print('The solution map looks like this:')
repaired_map, _ = repair_map(translate_gene(testSolution.gene, config['fitness_kwargs']['height'], config['fitness_kwargs']['width']))
plotMap(repaired_map)

del testSolution # to prevent haphazard copypasta

Notice that this time the `repair_and_test_map` function returned a list of values instead of a single value. The first value is negative average pac-man score like the previous assignments. The second value is the negative of the count of cells on the map surrounded by 3 walls (aka the negative count of dead ends). We want to maximize both of these objectives and assume that they are conflicting objectives since dead ends make great traps for pac-man. The number of dead ends should match with the visualization generated above.

Note: `repair_and_test_map` returns a list here because the config file defines the fitness keyword argument `minimize_dead_ends = True`.

Now that you've seen an example of calling the fitness function in a new way, write an `evaluate_population` function that takes as input a population and fitness_kwargs dictionary and populates the `objectives` and `log` member variables of all individuals in the population.

In [ ]:
# evaluate the population and assign objectives and logs as described above
def evaluate_population(population, fitness_kwargs):
    if fitness_kwargs.get('return_repair_count') != True:
        #GREEN deliverable logic goes here
        pass
    else:
        #RED deliverable logic goes here (if attempted)
        pass

In [ ]:
import statistics

examplePopulation = binaryGenotype.initialization(10, **config['initialization_kwargs'])
for individual in examplePopulation:
    individual.objectives = None

# calling your function to test things out
evaluate_population(examplePopulation, config['fitness_kwargs'])
print(f'Individuals with unassigned objectives: {len([individual.objectives for individual in examplePopulation if individual.objectives is None])}')
print(f'Number of fitness evaluations performed: {len([individual.objectives for individual in examplePopulation if individual.objectives is not None])}')
if config['fitness_kwargs'].get('minimize_dead_ends') == True:
    print(f'Average value of pac-man score objective: {statistics.mean([individual.objectives[0] for individual in examplePopulation])}')
    print(f'Average value of dead end objective: {statistics.mean([individual.objectives[1] for individual in examplePopulation])}')

del examplePopulation

## Multi-Objective Domination
As discussed in the course lectures, a prevalent way to assess solutions in an MOEA is to determine whether or not a solution *dominates* another solution in the population. Recall from the lecture that an individual `A` is said to dominate an individual `B` if and only if:
* `A` is no worse than `B` in all objectives *AND*
* `A` is strictly better than `B` in at least one objective

In the next cell, implement the function `dominates` that compares the `objectives` member variables of the input individuals returns `True` if `A` dominates `B` and `False` otherwise.

In [ ]:
# compare the individuals as described above
def dominates(A, B):
    pass

To evaluate your implementation of the `dominates` function, we're going to compare against the model answer from Question 11 of the second exam from Fall 2020 [(link here)](http://bonsai.auburn.edu/dtauritz/courses/ec/intro/2020fall/IntroECfs2020exam2key.pdf). The output generated by executing the following cell should match the domination table from part (a) of that problem.

In [ ]:
objectiveScores = [[8,2],
                   [4,1],
                   [2,3],
                   [1,2],
                   [9,1],
                   [4,7],
                   [2,5],
                   [1,3],
                   [10,7],
                   [5,5]]
expectation = [[2, 4],
              [],
              [4,8],
              [],
              [2],
              [2,3,4,7,8],
              [3,4,8],
              [4],
              [1,2,3,4,5,6,7,8,10],
              [2,3,4,7,8]]

examPopulation = binaryGenotype.initialization(len(objectiveScores), **config['initialization_kwargs'])

# assigning objectives this way for demo purposes only
for idx in range(len(objectiveScores)):
    examPopulation[idx].objectives = objectiveScores[idx]

print('ID\t Dominates')

# Note that this implementation of a domination table has some quirks so it matches the exam.
# Advise caution if you copy this code later because you should probably modify it.
for idx in range(len(examPopulation)):
    dominationList = list()
    for opponentIdx in range(len(examPopulation)):
        if idx == opponentIdx:
            continue
        if dominates(examPopulation[idx], examPopulation[opponentIdx]):
            dominationList.append(opponentIdx+1)
    print(f'{idx+1}\t{dominationList}', end='')
    if dominationList != expectation[idx]:
        print(f'\texpected {expectation[idx]}')
    else:
        print()
del examPopulation
del expectation

## Non-domination Sort
With the `dominates` function implemented, you can now implement the `non_domination_sort` function. This function takes as input a population and has three main steps:
1. Calculate a domination lookup table (this is technically optional but provides a drastic speedup)
2. Sort individuals into levels of non-domination with the algorithm performed in class and where level 0 is the Pareto front
3. Assign each individual a representative fitness to their `fitness` member variable equal to the number of non-domination levels minus the level of the individual

Note: you may implement additional helper functions to call within `non-domination_sort` so long as calling the `non_domination_sort` function produces the expected results.

In [ ]:
def non_domination_sort(population, crowding=False, **kwargs):
    # calculate domination table
    pass
    
    # sort into levels of non-domination
    pass
    
    # calculate representative fitness
    if crowding==False:
        # GREEN deliverable code goes here
        pass
    else:
        # YELLOW deliverable code for crowding goes here
        pass

We'll evaluate your `non_domination_sort` implementation using the same exam question we used to evaluate `dominates`. As such, your algorithm should generate `fitness` values that describe a non-domination sort with a similar result to that in the model answer.

In [ ]:
config = readConfig('./configs/green1d_config.txt', globalVars=globals(), localVars=locals())

examPopulation = binaryGenotype.initialization(len(objectiveScores), **config['initialization_kwargs'])

# assigning objectives this way for demo purposes only
for idx in range(len(objectiveScores)):
    examPopulation[idx].objectives = objectiveScores[idx]

# calling your function to test it out
non_domination_sort(examPopulation, **config['EA_configs'])

print(f'Individuals with unassigned fitness: {len([individual.fitness for individual in examPopulation if individual.fitness is None])}')
print(f'Number of fitness evaluations performed: {len([individual.fitness for individual in examPopulation if individual.fitness is not None])}')

fitnesses = set()

for individual in examPopulation:
    if individual.fitness is not None:
        # truncate the fitness values in case you've implemented the YELLOW deliverable
        individual.fitness = int(individual.fitness)
    fitnesses.add(individual.fitness)

fitnesses = sorted(list(fitnesses), reverse=True)
print('\nLevels of non-domination after adding all elements')
for levelIdx in range(len(fitnesses)):
    print(f'level {levelIdx}: {sorted([ind+1 for ind in range(len(examPopulation)) if examPopulation[ind].fitness == fitnesses[levelIdx]])}')

del examPopulation

## Assembling the MOEA
Now that you have `evaluate_fitness`, `dominates`, and `non_domination_sort` functions implemented, you can assemble your complete MOEA using the `baseEvolutionPopulation` population class you implemented in Assignment 1b. There are, however, some small differences from a single-objective EA that we'll walk you through in the following example.

In [ ]:
from baseEvolution import baseEvolutionPopulation

config = readConfig('./configs/green1d_config.txt', globalVars=globals(), localVars=locals())

# full initialization of your EA
exampleEA = baseEvolutionPopulation(**config['EA_configs'], **config)
evaluate_population(exampleEA.population, config['fitness_kwargs'])

# evaluate initial population
exampleEA.evaluations = len(exampleEA.population)
print(f'Number of fitness evaluations: {exampleEA.evaluations}')

Until this point, the EA has gone as expected. We've read a config, initialized the EA, and evaluated the initial population. Recall, however, that the fitness evaluation only assigns objective scores that can't directly be used as single-value fitness in evolution. To calculate a single fitness value and evolve as usual, we have to add a call to the new `non_domination_sort` before entering child generation (and parent selection).

In [ ]:
# sort population and assign representative fitness
non_domination_sort(exampleEA.population)

Once `non_domination_sort` has been called, the EA can generate children. Once the children are evaluated for objective scores and have been added to the population, we need to re-sort the population and re-calculate representative fitness before entering survival selection. As a rule of thumb, you'll need to re-calculate representative fitness using `non_domination_sort` before each function call that utilizes a selection algorithm.

In [ ]:
# generate children
children = exampleEA.generate_children()
evaluate_population(children, config['fitness_kwargs'])
exampleEA.evaluations += len(children)
print(f'Number of fitness evaluations: {exampleEA.evaluations}')

# re-sort modified population and assign representative fitness
non_domination_sort(exampleEA.population)

# perform survival selection
exampleEA.survival()

del exampleEA # to prevent haphazard copypasta

The calls to `non_domination_sort` are new additions to the EA cycle, but otherwise the MOEA cycle closely resembles that of previous assignments.

Now that you've implemented the necessary functions and the MOEA cycle has been demonstrated, implement a single run of your MOEA that searches for 2,000 fitness evaluations.

In [ ]:
number_evaluations = 2000

# You can parse different configuration files here as necessary
config = readConfig('./configs/green1d_config.txt', globalVars=globals(), localVars=locals())

# implement your EA here


Now that you've tested an implementation of a single run, implement code to perform 30 runs of your MOEA search that each contain 2,000 evaluations. For each generation of each run, log the mean and best values for all objectives in the current population as well as the number of fitness evaluations performed so far (including the initial population). Also for each run, record the objective scores and maps (either game logs or the genes) for all individuals in the Pareto front of the final generation. The maps and objective scores will be used for analysis in your report.

If you have attempted the YELLOW deliverable, you should record the diversity of the Pareto front from the final population of each run using a diversity metric of your choice.

In [ ]:
number_runs = 30
number_evaluations = 2000

# You can parse different configuration files here as necessary
config = readConfig('./configs/green1d_config.txt', globalVars=globals(), localVars=locals())

# implement your multi-run experiment here


## Report
Comparing multi-objective performance is a [known-difficult problem](http://lopez-ibanez.eu/hypervolume) we consider to be outside the scope of this class. The required analysis and statistics requirements vary per deliverable. See the assignment description for more details.